# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import warnings 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
datafile =pd.read_csv ("../output_data/mycitydata.csv")

datafile.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,4030556,Rikitea,78,PF,1603343901,73,-23.12,-134.97,74.12,2.59
1,1649150,Bengkulu,100,ID,1603343777,72,-3.80,102.27,81.01,10.16
2,4465088,Elizabeth City,90,US,1603343938,100,36.29,-76.25,66.20,3.20
3,3421765,Nanortalik,36,GL,1603343906,81,60.14,-45.24,34.07,36.06
4,5847411,Kahului,20,US,1603343917,58,20.89,-156.47,82.40,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure Heatmap
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = datafile[["Lat", "Lng"]].astype(float)
humidity = datafile["Humidity"].astype(float)

fig = gmaps.figure()
# create map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
# add layer
fig.add_layer(heat_layer)
# show map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
new_df = pd.DataFrame(datafile, columns = ['City', 'Max Temp','Cloudiness','Humidity','Lat','Lng','Country'])
new_df.head()

,City,Max Temp,Cloudiness,Humidity,Lat,Lng,Country
0,Rikitea,74.12,78,73,-23.12,-134.97,PF
1,Bengkulu,81.01,100,72,-3.80,102.27,ID
2,Elizabeth City,66.20,90,100,36.29,-76.25,US
3,Nanortalik,34.07,36,81,60.14,-45.24,GL
4,Kahului,82.40,20,58,20.89,-156.47,US


In [5]:
# City that has the weather of my choice
hotel_df = new_df.loc[(new_df['Max Temp']>56) & (new_df['Max Temp']<80) & (new_df['Cloudiness']>0) 
                          & (new_df['Cloudiness']<40) & (new_df['Humidity']<30)]

hotel_df

,City,Max Temp,Cloudiness,Humidity,Lat,Lng,Country
63,Saint George,66.20,1,16,37.10,-113.58,US
188,Sun City West,78.01,1,22,33.66,-112.34,US
332,Biu,69.01,1,27,10.61,12.19,NG
530,Taoudenni,74.37,17,20,22.68,-3.98,ML
614,Harar,69.51,16,29,9.31,42.13,ET
633,Linguère,77.85,23,24,15.40,-15.12,SN
646,Guarujá,73.99,6,26,-23.99,-46.26,BR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Hotel variable 
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\nikDev\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Max Temp,Cloudiness,Humidity,Lat,Lng,Country,Hotel Name
63,Saint George,66.20,1,16,37.10,-113.58,US,
188,Sun City West,78.01,1,22,33.66,-112.34,US,
332,Biu,69.01,1,27,10.61,12.19,NG,
530,Taoudenni,74.37,17,20,22.68,-3.98,ML,
614,Harar,69.51,16,29,9.31,42.13,ET,
633,Linguère,77.85,23,24,15.40,-15.12,SN,
646,Guarujá,73.99,6,26,-23.99,-46.26,BR,


In [7]:
for index, row in hotel_df.iterrows():
    lat =  row["Lat"]
    lng =  row["Lng"]
    
    #check to see if coor is pulled
    #print((f"{lat},{lng}"))
    
    # geocoordinates
    target_coordinates = (f"{lat},{lng}")
    #target_search = "Chinese"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
            "location":target_coordinates,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
    }
    
    
    #print(params)
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using my params dictionary
    hotel_resp = requests.get(base_url, params=params).json()

    # print pretty data
    #print(json.dumps(hotel_resp, indent=4, sort_keys=True))
                           
    try: 
  
       hotel_df.loc[index,'Hotel Name'] =hotel_resp["results"][0]["name"]
    except: 
       print("Hotel Not Found")
print("------End Run Code---------")


C:\Users\nikDev\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Hotel Not Found
------End Run Code---------


In [8]:
# Clean dataset 
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

C:\Users\nikDev\anaconda3\envs\PythonData\lib\site-packages\pandas\core\series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\Users\nikDev\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Max Temp,Cloudiness,Humidity,Lat,Lng,Country,Hotel Name
63,Saint George,66.20,1,16,37.10,-113.58,US,Best Western Plus Abbey Inn
188,Sun City West,78.01,1,22,33.66,-112.34,US,Windmill All Suites Surprise
332,Biu,69.01,1,27,10.61,12.19,NG,EYN LCC MANDARAGIRAU
614,Harar,69.51,16,29,9.31,42.13,ET,Amir Cultural Harari Guesthouse
633,Linguère,77.85,23,24,15.40,-15.12,SN,Hôtel Amadou Seck
646,Guarujá,73.99,6,26,-23.99,-46.26,BR,Charme Hotel Guarujá


In [9]:
#Configure My Weather - Hotel Heatmap
# Create Hotel symbol layer
gmaps.configure(api_key=g_key)



# Store 'Lat' and 'Lng' into  locations 
locations = hotel_df[["Lat", "Lng"]].astype(float)
humidity = hotel_df["Humidity"].astype(float)


figMap = gmaps.figure()
# create map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)


# add layer
figMap.add_layer(heat_layer)

# show map
figMap

Figure(layout=FigureLayout(height='420px'))

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [29]:
# Add marker layer ontop of heat map
figMap = gmaps.figure(map_type='SATELLITE')
hotelmarkers = gmaps.marker_layer(locations,info_box_content=hotel_info)

figMap.add_layer(hotelmarkers)

figMap

Figure(layout=FigureLayout(height='420px'))